# From Qiskit to Quantum Hardware with Arvak & Nathan

**What this demo shows:** The complete experience of a Qiskit developer discovering Arvak, getting AI-powered research feedback from Nathan, and running circuits on real quantum hardware (IBM Torino & Scaleway/IQM Garnet).

---

## The Scenario

You're a Qiskit developer working on a portfolio optimization problem. You've built your QAOA circuit, but you're not sure:
- Is this circuit any good?
- What does the literature say about this approach?
- How will it perform on real hardware?
- Can it be optimized further?

Let's find out.

## Step 1: Installation

You already have Qiskit. Adding Arvak is one line:

In [ ]:
# One-liner — includes Qiskit integration + Nathan research optimizer
# pip install arvak[qiskit]

# Or if you want everything (Cirq, PennyLane, Qrisp, Nathan, Jupyter extras):
# pip install arvak[all]

## Step 2: Write Your Circuit (Normal Qiskit — Nothing Changes)

This is the key point: **you don't rewrite anything.** Your existing Qiskit code works as-is.

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

# ── Your existing QAOA circuit for a 4-asset portfolio optimization ──

n_qubits = 4  # 4 assets
p = 1         # 1 QAOA layer (you're starting simple)

gamma = 0.8   # problem parameter
beta = 0.4    # mixer parameter

qc = QuantumCircuit(n_qubits, n_qubits)

# Initial superposition
for i in range(n_qubits):
    qc.h(i)

# Problem unitary (diagonal cost Hamiltonian for portfolio)
for i in range(n_qubits):
    qc.rz(2 * gamma * 0.5, i)        # linear terms (expected returns)
for i in range(n_qubits - 1):
    qc.cx(i, i + 1)
    qc.rz(2 * gamma * 0.3, i + 1)    # quadratic terms (covariance)
    qc.cx(i, i + 1)

# Mixer unitary
for i in range(n_qubits):
    qc.rx(2 * beta, i)

# Measure
qc.measure(range(n_qubits), range(n_qubits))

print(f"Circuit: {n_qubits} qubits, depth {qc.depth()}, {qc.size()} gates")
qc.draw('mpl')

## Step 3: Ask Nathan — "Is This Any Good?"

Before burning quantum hardware time, let's ask Nathan to analyze the circuit.
Nathan searches 1,700+ papers and gives you concrete, actionable feedback.

In [ ]:
import arvak

# Hand Nathan your Qiskit circuit — it handles the conversion automatically
report = arvak.nathan.analyze(qc)

# In Jupyter, this renders as a styled dark-mode card automatically
report

### What Nathan Tells You

The report includes:

| Field | What it means |
|-------|---------------|
| **Problem type** | Nathan detected this is a QAOA circuit |
| **Suitability score** | 0.0–1.0 — how well this maps to near-term quantum hardware |
| **Recommended algorithm** | What the literature says works best for this problem size |
| **Hardware fit** | How this circuit matches a specific backend's topology |
| **Papers** | Relevant arXiv papers with URLs |
| **Suggestions** | Concrete optimizations — with QASM3 code you can apply directly |

Let's dig into the details:

In [ ]:
# ── Inspect Nathan's findings ──

print(f"Problem type:    {report.problem_type}")
print(f"Suitability:     {report.suitability:.0%}")
print(f"Best algorithm:  {report.recommended_algorithm}")
print(f"Hardware fit:    {report.hardware_fit}")
print(f"Est. error rate: {report.estimated_error_rate}")
print(f"Rec. shots:      {report.recommended_shots}")
print()

# Circuit stats Nathan extracted
if report.circuit:
    print(f"Circuit stats:")
    print(f"  Qubits: {report.circuit.num_qubits}")
    print(f"  Gates:  {report.circuit.total_gates}")
    print(f"  Depth:  {report.circuit.depth}")
    print(f"  Gates:  {report.circuit.gate_breakdown}")
    print(f"  Pattern: {report.circuit.detected_pattern}")

In [ ]:
# ── What papers should you read? ──

print("Relevant papers:")
print("=" * 60)
for p in report.papers:
    print(f"\n📄 {p.title}")
    print(f"   {p.arxiv_url}")
    print(f"   Algorithm: {p.algorithm}")
    print(f"   Why: {p.relevance}")

In [ ]:
# ── What should you change? ──

print("Optimization suggestions:")
print("=" * 60)
for i, s in enumerate(report.suggestions, 1):
    print(f"\n[{i}] {s.title}  (impact: {s.impact})")
    print(f"    {s.description}")
    if s.qasm3:
        print(f"    → QASM3 rewrite available: s.circuit gives you an arvak.Circuit")

## Step 4: Ask Nathan a Follow-Up Question

Nathan also has a chat mode. You can ask it anything about quantum algorithms.

In [ ]:
# Chat with Nathan — grounded in 1,700+ papers
resp = arvak.nathan.chat(
    "For a 4-asset portfolio optimization with QAOA, should I use p=1 or p=3 layers? "
    "What's the trade-off on near-term hardware?"
)

# Renders as styled card in Jupyter
resp

In [ ]:
# Follow-up with context from previous answer
resp2 = arvak.nathan.chat(
    "What about using CVaR-QAOA instead of standard QAOA for this problem?",
    context=resp.message
)
resp2

## Step 5: Apply a Suggestion & Test Locally

Nathan suggested improvements. Let's apply one and test on the local simulator first.

In [ ]:
# ── Apply Nathan's suggested circuit (if it has a QASM3 rewrite) ──

if report.suggestions and report.suggestions[0].qasm3:
    # Nathan's suggestion comes as a live arvak.Circuit
    optimized = report.suggestions[0].circuit
    print(f"Optimized circuit: {optimized.num_qubits} qubits, depth {optimized.depth()}")
else:
    # If no QASM3 rewrite, let's build an improved version manually
    # (e.g., increase QAOA layers from p=1 to p=2 as Nathan likely suggested)
    qc_v2 = QuantumCircuit(n_qubits, n_qubits)
    
    for i in range(n_qubits):
        qc_v2.h(i)
    
    # Two QAOA layers instead of one
    for layer, (g, b) in enumerate([(0.8, 0.4), (0.6, 0.3)]):
        for i in range(n_qubits):
            qc_v2.rz(2 * g * 0.5, i)
        for i in range(n_qubits - 1):
            qc_v2.cx(i, i + 1)
            qc_v2.rz(2 * g * 0.3, i + 1)
            qc_v2.cx(i, i + 1)
        for i in range(n_qubits):
            qc_v2.rx(2 * b, i)
    
    qc_v2.measure(range(n_qubits), range(n_qubits))
    optimized_qiskit = qc_v2
    print(f"Improved circuit: {n_qubits} qubits, depth {qc_v2.depth()}, {qc_v2.size()} gates")

In [ ]:
# ── Test on Arvak's local simulator (Rust statevector, no network needed) ──

from arvak.integrations.qiskit import ArvakProvider

provider = ArvakProvider()
sim = provider.get_backend('sim')  # Local Rust simulator

# Run the original circuit
job_v1 = sim.run(qc, shots=4096)
counts_v1 = job_v1.result().get_counts()

print("Original (p=1):")
for bitstring, count in sorted(counts_v1.items(), key=lambda x: -x[1])[:5]:
    print(f"  |{bitstring}⟩  {count:4d} shots  ({count/4096:.1%})")

In [ ]:
# ── Compare: run the improved circuit ──

job_v2 = sim.run(optimized_qiskit if 'optimized_qiskit' in dir() else qc, shots=4096)
counts_v2 = job_v2.result().get_counts()

print("Improved (p=2):")
for bitstring, count in sorted(counts_v2.items(), key=lambda x: -x[1])[:5]:
    print(f"  |{bitstring}⟩  {count:4d} shots  ({count/4096:.1%})")

## Step 6: Look Under the Hood — What Arvak Does Differently

Let's see the Arvak native API and how your Qiskit circuit converts to QASM3.
Compilation (layout, routing, basis translation) happens automatically inside
`backend.run()` — the Rust compiler handles it at ~100µs per circuit.

In [ ]:
import arvak
import time

# Convert your Qiskit circuit to Arvak's native representation
qiskit_int = arvak.get_integration('qiskit')
arvak_circuit = qiskit_int.to_arvak(qc)

print(f"Arvak circuit: {arvak_circuit.num_qubits} qubits, {arvak_circuit.size()} ops, depth {arvak_circuit.depth()}")
print()

# See the QASM3 that Arvak generates
qasm_out = arvak.to_qasm(arvak_circuit)
print("QASM3 representation:")
print(qasm_out)
print()

# Show available compilation targets
print("Available basis gate sets:")
for name, factory in [("IBM", arvak.BasisGates.ibm), ("IQM", arvak.BasisGates.iqm), ("Universal", arvak.BasisGates.universal)]:
    bg = factory()
    print(f"  {name}: {bg}")
print()
print("Available coupling maps:")
print(f"  Linear(4):  {arvak.CouplingMap.linear(4)}")
print(f"  Star(5):    {arvak.CouplingMap.star(5)}")
print(f"  Full(4):    {arvak.CouplingMap.full(4)}")

In [ ]:
# ── Speed test: build & simulate 1,000 circuit variations ──

circuits = []
for gamma_val in np.linspace(0.1, 2.0, 1000):
    c = arvak.Circuit("sweep", num_qubits=4)
    for i in range(4):
        c.h(i)
    for i in range(4):
        c.rz(2 * gamma_val * 0.5, i)
    for i in range(3):
        c.cx(i, i + 1)
    c.measure_all()
    circuits.append(c)

start = time.perf_counter()
results = [arvak.run_sim(c, shots=100) for c in circuits]
elapsed = time.perf_counter() - start

print(f"Built & simulated 1,000 circuits in {elapsed:.2f}s ({elapsed/1000*1e6:.0f}µs per circuit)")
print(f"Sample result (gamma=0.1): {results[0]}")
print(f"Sample result (gamma=2.0): {results[-1]}")

## Step 7: Shoot Qubits — Run on Real Quantum Hardware

Arvak supports multiple hardware backends. Pick your target:

### Option A: IBM Torino (133-qubit Heron)
```bash
export IBM_API_KEY="your-ibm-cloud-api-key"
export IBM_SERVICE_CRN="your-service-crn"
```

### Option B: Scaleway QaaS / IQM Garnet (20-qubit)
```bash
export SCALEWAY_SECRET_KEY="your-secret-key"
export SCALEWAY_PROJECT_ID="your-project-id"
export SCALEWAY_SESSION_ID="your-session-id"    # from Scaleway console
export SCALEWAY_PLATFORM="QPU-GARNET-20PQ"       # or QPU-EMERALD-54PQ
```

In [ ]:
# ── Option A: Submit to IBM Quantum via Arvak's Qiskit provider ──

from arvak.integrations.qiskit import ArvakProvider
provider = ArvakProvider()

# Simulate locally first (always works, no credentials needed)
sim = provider.get_backend('sim')
job_sim = sim.run(qc, shots=4096)
counts_sim = job_sim.result().get_counts()

print("Local simulation results:")
print("=" * 40)
for bitstring, count in sorted(counts_sim.items(), key=lambda x: -x[1])[:8]:
    bar = '█' * int(count / 4096 * 40)
    print(f"  |{bitstring}⟩  {count:4d}  ({count/4096:5.1%})  {bar}")

In [ ]:
# ── Option B: Submit to Scaleway/IQM via Arvak CLI ──
# The Scaleway adapter runs through the CLI (Rust binary)
# Export your circuit to QASM3, then fire it at IQM Garnet

import subprocess, json, os, tempfile

# Save circuit as QASM3
qasm_str = arvak.to_qasm(qiskit_int.to_arvak(qc))
qasm_file = os.path.join(tempfile.gettempdir(), "portfolio_qaoa.qasm")
with open(qasm_file, "w") as f:
    f.write(qasm_str)
print(f"Circuit saved to {qasm_file}")
print(f"QASM3 ({len(qasm_str)} bytes):\n{qasm_str[:300]}...")
print()

# Check which backends are available
has_scaleway = all(os.environ.get(k) for k in ["SCALEWAY_SECRET_KEY", "SCALEWAY_PROJECT_ID", "SCALEWAY_SESSION_ID"])
has_ibm = os.environ.get("IBM_API_KEY") and os.environ.get("IBM_SERVICE_CRN")

print(f"IBM Quantum configured:      {'YES' if has_ibm else 'NO'}")
print(f"Scaleway/IQM configured:     {'YES' if has_scaleway else 'NO'}")

if has_scaleway:
    print(f"\nScaleway platform: {os.environ.get('SCALEWAY_PLATFORM', 'QPU-GARNET-20PQ')}")
    print(f"Session ID: {os.environ['SCALEWAY_SESSION_ID'][:12]}...")
    print("\nTo submit via CLI:")
    print(f"  arvak run {qasm_file} --backend scaleway --shots 4096")
elif has_ibm:
    print("\nTo submit via CLI:")
    print(f"  arvak run {qasm_file} --backend ibm --shots 4096")
else:
    print("\nNo hardware credentials found. Set env vars above, then re-run.")

## Step 8: Ask Nathan About Your Results

Now that we have real hardware results, let's ask Nathan what to make of them.

In [ ]:
# Ask Nathan specifically about hardware performance
hw_report = arvak.nathan.analyze(qc, backend="ibm_marrakesh")

print(f"Hardware fit:      {hw_report.hardware_fit}")
print(f"Est. error rate:   {hw_report.estimated_error_rate}")
print(f"Recommended shots: {hw_report.recommended_shots}")
print()

# The full report renders beautifully in Jupyter
hw_report

In [ ]:
# Ask about error mitigation options
mitigation = arvak.nathan.chat(
    "My QAOA circuit on ibm_marrakesh shows noisy results. "
    "What error mitigation techniques work best for QAOA on IBM Heron processors? "
    "I have 4 qubits and p=1 layer.",
)
mitigation

## Step 9: Compare Simulator vs. Hardware

Let's visualize the difference between ideal simulation and real hardware.

In [ ]:
import matplotlib.pyplot as plt

# Get all bitstrings present in either result
all_bits = sorted(set(list(counts_v1.keys()) + list(counts_hw.keys())))

sim_vals = [counts_v1.get(b, 0) / 4096 for b in all_bits]
hw_vals = [counts_hw.get(b, 0) / 4096 for b in all_bits]

x = np.arange(len(all_bits))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(x - width/2, sim_vals, width, label='Arvak Simulator', color='#6366f1')
ax.bar(x + width/2, hw_vals, width, label='IBM Marrakesh (Hardware)', color='#f59e0b')

ax.set_xlabel('Measurement outcome')
ax.set_ylabel('Probability')
ax.set_title('QAOA Portfolio Optimization: Simulator vs. Real Hardware')
ax.set_xticks(x)
ax.set_xticklabels([f'|{b}⟩' for b in all_bits], rotation=45)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Step 10: The Full Picture — What Just Happened

Here's the complete flow you just experienced:

```
┌─────────────────────────────────────────────────────────────────┐
│  Your Qiskit Code (unchanged)                                   │
│  qc = QuantumCircuit(4, 4)                                      │
│  qc.h(0); qc.cx(0, 1); ...                                     │
└──────────────────────┬──────────────────────────────────────────┘
                       │
                       ▼
┌──────────────────────────────────────────────────────────────────┐
│  Nathan: "Here's what the literature says..."                    │
│  • Problem type: QAOA portfolio optimization                     │
│  • Suitability: 72%                                              │
│  • Suggestion: increase to p=2, use CVaR objective               │
│  • Papers: 5 relevant arXiv papers                               │
│  • QASM3 rewrite: ready to apply                                 │
└──────────────────────┬──────────────────────────────────────────┘
                       │
                       ▼
┌──────────────────────────────────────────────────────────────────┐
│  Arvak Compiler (Rust, ~100µs/circuit)                           │
│  • Layout mapping (logical → physical qubits)                    │
│  • SWAP routing for hardware topology                            │
│  • Basis translation (→ RZ/SX/CX for IBM, PRX/CZ for IQM)      │
│  • Gate cancellation & optimization                              │
└──────────┬───────────────────────────────┬──────────────────────┘
           │                               │
           ▼                               ▼
┌─────────────────────────┐  ┌────────────────────────────────────┐
│  IBM Torino              │  │  Scaleway / IQM Garnet             │
│  133-qubit Heron         │  │  20-qubit (or 54-qubit Emerald)    │
│  RZ, SX, CX native      │  │  PRX, CZ native                   │
└─────────────────────────┘  └────────────────────────────────────┘
```

### What's Different From Pure Qiskit?

| | Qiskit alone | Qiskit + Arvak |
|---|---|---|
| **Your code** | Same | Same (zero changes) |
| **Research help** | Manual paper search | Nathan: 1,700+ papers, instant |
| **Compilation** | Python transpiler | Rust compiler (~100µs) |
| **Hardware** | IBM only | IBM, IQM (via Scaleway), Neutral-Atom, CUDA-Q |
| **Optimization suggestions** | None | With QASM3 rewrites |
| **Privacy** | N/A | Circuit anonymized before analysis |
| **IQM access** | Not supported | Native PRX/CZ compilation + Scaleway QaaS |

---

## Bonus: Cross-Framework in 3 Lines

Your Qiskit circuit can run on any backend Arvak supports — without rewriting.

In [ ]:
# Your Qiskit circuit → IQM hardware (if you have access)
# iqm_backend = provider.get_backend('iqm_garnet')
# job = iqm_backend.run(qc, shots=4096)

# Your Qiskit circuit → Cirq (Google's framework)
cirq_int = arvak.get_integration('cirq')
if cirq_int:
    arvak_circ = qiskit_int.to_arvak(qc)
    cirq_circ = cirq_int.from_arvak(arvak_circ)
    print(f"Same circuit, now in Cirq: {type(cirq_circ)}")

# Your Qiskit circuit → PennyLane (for QML workflows)
pl_int = arvak.get_integration('pennylane')
if pl_int:
    pl_circ = pl_int.from_arvak(arvak_circ)
    print(f"Same circuit, now in PennyLane: {type(pl_circ)}")

## Getting Started

```bash
pip install arvak[qiskit]
```

That's it. Your existing Qiskit code works. Nathan is ready. Hardware is one `get_backend()` call away.

**Links:**
- Documentation: [arvak.io](https://arvak.io)
- Nathan web interface: [arvak.io/nathan](https://arvak.io/nathan)
- GitHub: [github.com/hiq-lab/arvak](https://github.com/hiq-lab/arvak)